## Test of the targeted controller for std1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from loguru import logger

In [3]:
vimms_folder = '/Users/simon/git/vimms'
sys.path.append(vimms_folder)

In [4]:
from vimms.Chemicals import ChemicalMixtureFromMZML
from vimms.Controller import create_targets_from_toxid, TargetedController
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Environment import Environment
from vimms.Common import POSITIVE, set_log_level_warning
from vimms.Roi import RoiParams

## Set the path to the experimental folder to get the mzML

In [5]:
experimental_results_folder = '/Users/simon/clms_er'
data_folder = os.path.join(experimental_results_folder,'20200904_DIA_test','02_DIA_experiments_stdmix')

In [6]:
standard = 'std1'
mzml_file = os.path.join(data_folder,'{}_Fullscan.mzML'.format(standard))

## Setup RIO parameters for the mzml -> chemicals process and create chemicals

In [7]:
rp = RoiParams(min_intensity=500)
cm = ChemicalMixtureFromMZML(mzml_file, roi_params=rp)
dataset = cm.sample(None, 2)

2020-09-11 12:38:04.290 | DEBUG    | vimms.Chemicals:_extract_rois:794 - Extracted 18474 good ROIs from /Users/simon/clms_er/20200904_DIA_test/02_DIA_experiments_stdmix/std1_Fullscan.mzML


## Load the toxid file and make the target list

In [9]:
toxid_file = os.path.join(vimms_folder,'tests','fixtures','StdMix1_pHILIC_Current.csv')
target_list = create_targets_from_toxid(toxid_file, rt_delta=5*60., mz_delta=5, polarity_filter=['+'])
print(len(target_list))

111


## Setup the controller, mass spec and environment

In [10]:
ce_values = [10, 20, 30]
n_replicates = 2
ionisation_mode = POSITIVE
controller = TargetedController(target_list, ce_values, n_replicates=n_replicates, limit_acquisition=False, min_ms1_intensity=500)
ms = IndependentMassSpectrometer(ionisation_mode, dataset, None)
env = Environment(ms, controller, 0, 1600)

## Run the simulation

In [11]:
set_log_level_warning()
env.run()

(1600.200s) ms_level=1: 100%|█████████▉| 1599.8000000002053/1600 [00:59<00:00, 26.97it/s]


## Summarise the targets -- what has been hit

In [12]:
controller.summarise_activity(print)

Summary of targets
Melatonin[M+H]+ (m/z: 233.12728857349967->233.12961985804182, rt: -42.60000000000002->557.4)
	10 -> 37 scans
	20 -> 37 scans
	30 -> 37 scans
Melatonin[M+K]+ (m/z: 271.0829804420914->271.08569128545, rt: -42.60000000000002->557.4)
	10 -> 9 scans
	20 -> 9 scans
	30 -> 9 scans
Melatonin[M+Na]+ (m/z: 255.1091229117784->255.11167401576301, rt: -42.60000000000002->557.4)
	10 -> 15 scans
	20 -> 15 scans
	30 -> 15 scans
Phenylhydrazine[M+H]+ (m/z: 109.07547933866714->109.07657009891433, rt: -12.0->588.0)
	10 -> 88 scans
	20 -> 88 scans
	30 -> 87 scans
Phenylhydrazine[M+K]+ (m/z: 147.03117120725886->147.03264152632255, rt: -12.0->588.0)
	10 -> 0 scans
	20 -> 0 scans
	30 -> 0 scans
Phenylhydrazine[M+Na]+ (m/z: 131.05731367694585->131.05862425663554, rt: -12.0->588.0)
	10 -> 0 scans
	20 -> 0 scans
	30 -> 0 scans
Nicotinate[M+H]+ (m/z: 124.03868466028645->124.03992505333501, rt: 137.39999999999998->737.4)
	10 -> 93 scans
	20 -> 93 scans
	30 -> 93 scans
Nicotinate[M+K]+ (m/z: 161

In [133]:
env.write_mzML(None, 'test.mzML')